# 2A.algo - Plus proches voisins en grande dimension

La méthodes des [plus proches voisins](https://fr.wikipedia.org/wiki/Recherche_des_plus_proches_voisins) est un algorithme assez simple. Que se passe-t-il quand la dimension de l'espace des features augmente ? Comment y remédier ? Le profiling [memory_profiler](https://pypi.python.org/pypi/memory_profiler) ou [cprofile](https://docs.python.org/3.7/library/profile.html#module-cProfile) sont des outils utiles pour savoir où le temps est perdu. 

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## Q1 : k-nn : mesurer la performance

In [2]:
from sklearn.datasets import make_classification
datax, datay = make_classification(10000, n_features=10, n_classes=3, 
                                   n_clusters_per_class=2, n_informative=8)
datax[:3]

array([[-0.8475772 , -2.32538375,  2.85493495,  0.80844826, -0.22859889,
        -1.04841583,  0.02968567,  0.64623341,  0.80613674, -2.23389406],
       [-0.98432181, -0.06661461,  7.75513731, -0.68528612,  2.91266715,
        -2.42866215, -1.30340144, -2.10535336,  2.30057811, -0.16914582],
       [ 2.5080994 ,  0.78644825,  2.64918709,  1.47316878, -6.35328966,
        -0.82007342, -0.08550633, -5.23436533, -0.56694263, -2.1252314 ]])

In [3]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(5, algorithm="brute")
model.fit(datax, datay)

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [4]:
model.predict(datax)

array([2, 1, 0, ..., 1, 0, 0])

In [5]:
%timeit model.predict(datax)

2.78 s ± 27.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [1]:
import numpy
import os
path = os.path.normpath(os.path.join(numpy.__file__, '..', '..'))
print(path)

/home/kovalsky/.local/lib/python3.10/site-packages


In [2]:
import cProfile
import pstats
from io import StringIO
pr = cProfile.Profile()
pr.enable()
model.predict(datax)
pr.disable()
s = StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats('cumulative')
ps.print_stats()
res = s.getvalue().replace(path, '').replace("\\", "/").replace(" /", " ")
print('\n'.join(res.split('\n')[:50]))

NameError: name 'model' is not defined

Etudier l'évolution du temps de prédiction en fonction du nombre d'observations, de la dimension, du nombre de classes ? Qu'en déduisez-vous ? Le code sur GitHub :

* [predict](https://github.com/scikit-learn/scikit-learn/blob/ef5cb84a/sklearn/neighbors/classification.py#L129)
* [kneighbors](https://github.com/scikit-learn/scikit-learn/blob/ef5cb84a805efbe4bb06516670a9b8c690992bd7/sklearn/neighbors/base.py#L273)
* [pairwise_distance](https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/metrics/pairwise.py#L1141)

## Q2 : k-nn avec sparse features

On recommence cette mesure de temps mais en créant des jeux de données [sparses](https://fr.wikipedia.org/wiki/Matrice_creuse).

## Q3 : Imaginez une façon d'aller plus vite ?

Aller plus vite veut parfois dire perdre un peu en performance et dans notre cas, on veut accélérer la prédiction.